### Домашнее задание №6.

***1. Взять любой набор данных для бинарной классификации.***

***2. Обучить любой классификатор.***

***3. Разделить ваш набор данных на два множества: P (positives) и U (unlabeled). 
Причем брать нужно не все положительные примеры (класс 1), а только лишь часть.***

***4. Применить random negative sampling для построения классификатора в новых условиях.***

***5. Сравнить качество с решением из пункта 3 (построить отчет - таблицу метрик).***

In [1]:
# Загрузим необходимые библиотеки:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score
from catboost import CatBoostClassifier

In [2]:
import warnings
warnings.simplefilter('ignore')

In [3]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [4]:
df = pd.read_csv('./cardio_case.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


***Посмотрим на количество данных в датасете***

In [5]:
df.shape

(70000, 13)

***Проверим датасет на пропуски и на тип данных (для корректной работы модели):***

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           70000 non-null  int64  
 1   age          70000 non-null  int64  
 2   gender       70000 non-null  int64  
 3   height       70000 non-null  int64  
 4   weight       70000 non-null  float64
 5   ap_hi        70000 non-null  int64  
 6   ap_lo        70000 non-null  int64  
 7   cholesterol  70000 non-null  int64  
 8   gluc         70000 non-null  int64  
 9   smoke        70000 non-null  int64  
 10  alco         70000 non-null  int64  
 11  active       70000 non-null  int64  
 12  cardio       70000 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 6.9 MB


***Посмотрим на соотношение классов:***

In [7]:
df['cardio'].value_counts()

0    35021
1    34979
Name: cardio, dtype: int64

***Разобъем выборку на тренировочную и тестовую и обучим модель. 
Возьмем модель градиентного бустинга CatBoostClassifier.***

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='cardio'), df['cardio'], random_state=0)

***Обучим модель и сделаем предсказания:***

In [9]:
%%time
model = CatBoostClassifier(verbose=False)

model.fit(X_train, y_train)
cat_predict = model.predict(X_test)

Wall time: 20.3 s


***Чтобы удобнее было сравнивать результаты работы моделей - в финале выведем их в виде таблицы. Для этого создадим словарь, который будет собирать наши данные.***

In [10]:
models_results = {
    'model': [],
    'f1_score': [],
    'roc_auc': [],
    'recall': [],
    'precision': []
}

***Для проверки качества моделей и сбора результатов в таблицу напишем функцию:***

In [11]:
# Проверим качество:
def evaluate_results(y_test, y_predict, model):
    print('Classification results:')
    models_results['model'].append(model)
    f1 = f1_score(y_test, y_predict)
    models_results['f1_score'].append(f1)
    print(f"f1: {f1 * 100.0:.2f}%") 
    roc = roc_auc_score(y_test, y_predict)
    models_results['roc_auc'].append(roc)
    print(f"roc: {roc * 100.0:.2f}%") 
    rec = recall_score(y_test, y_predict, average='binary')
    models_results['recall'].append(rec)                    
    print(f"recall: {rec * 100.0:.2f}%") 
    prc = precision_score(y_test, y_predict, average='binary')
    models_results['precision'].append(prc)                   
    print(f"precision: {prc * 100.0:.2f}%" ) 

    
evaluate_results(y_test, cat_predict, 'CatBoostClassifier')

Classification results:
f1: 71.95%
roc: 73.34%
recall: 68.85%
precision: 75.35%


***Для самоконтроля обучим на этих данных другую модель и тоже проверим качество:***

In [12]:
%%time
import xgboost as xgb

model = xgb.XGBClassifier()

model.fit(X_train, y_train)
xgb_predict = model.predict(X_test)

[17:11:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Wall time: 3.84 s


In [13]:
evaluate_results(y_test, xgb_predict, 'XGBClassifier')

Classification results:
f1: 71.73%
roc: 73.12%
recall: 68.66%
precision: 75.08%


***Результаты у обеих моделей очень похожи, но CatBoost показала себя немного лучше. В дальнейшем для работы будем использовать её, а XGB для контроля.***

***Предположим, что нам неизвестны негативы и часть позитивов.***

In [14]:
data = df.copy()

In [15]:
mod_data = data.copy()

# get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:, -1].values == 1)[0]

# shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
perc = 0.25
pos_sample_len = int(np.ceil(perc * len(pos_ind)))

print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 8745/34979 as positives and unlabeling the rest


***Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1).***

In [16]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    61255
 1     8745
Name: class_test, dtype: int64


***У нас получилось 8745 позитивных примеров и 61255 без разметки.***

In [17]:
mod_data.head(10)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,class_test
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0,-1
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1,-1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1,-1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0,-1
5,8,21914,1,151,67.0,120,80,2,2,0,0,0,0,-1
6,9,22113,1,157,93.0,130,80,3,1,0,0,1,0,-1
7,12,22584,2,178,95.0,130,90,3,3,0,0,1,1,-1
8,13,17668,1,158,71.0,110,70,1,1,0,0,1,0,-1
9,14,19834,1,164,68.0,110,60,1,1,0,0,0,0,-1


***RNS - Помним, что ('cardio') содержит целевой признак, который будем использовать для оценки качества
Отделими [:-2] 'cardio' как истиный класс для проверки, и [:-1] 'class_test' как данные для входной разметки PUL.***

In [18]:
mod_data = mod_data.sample(frac=1)


data_N = mod_data[mod_data['class_test'] == -1]
data_P = mod_data[mod_data['class_test'] == 1]

neg_sample = data_N[:data_P.shape[0]]
sample_test = data_N[data_P.shape[0]:]
pos_sample = data_P.copy()

print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(8745, 14) (8745, 14)


In [19]:
%%time
model = CatBoostClassifier(verbose=False)
sample_train.loc[sample_train['class_test'] == -1, 'class_test'] = 0

model.fit(sample_train.drop(columns=['class_test', 'cardio']), 
          sample_train['class_test'])

cat_1_predict = model.predict(sample_test.drop(columns=['class_test', 'cardio']))
evaluate_results(sample_test['cardio'], cat_1_predict, 'CatBoostClassifier_PUL')

Classification results:
f1: 68.03%
roc: 72.26%
recall: 66.89%
precision: 69.21%
Wall time: 12 s


In [20]:
%%time
model = xgb.XGBClassifier()
sample_train.loc[sample_train['class_test'] == -1, 'class_test'] = 0

model.fit(sample_train.drop(columns=['class_test', 'cardio']), 
          sample_train['class_test'])

xgb_1_predict = model.predict(sample_test.drop(columns=['class_test', 'cardio']))
evaluate_results(sample_test['cardio'], xgb_1_predict, 'XGBClassifier_PUL')

[17:11:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Classification results:
f1: 64.69%
roc: 69.30%
recall: 63.87%
precision: 65.52%
Wall time: 1.58 s


In [21]:
pd.DataFrame(data=models_results)

,model,f1_score,roc_auc,recall,precision
0,CatBoostClassifier,0.719523,0.733412,0.688479,0.753499
1,XGBClassifier,0.717294,0.731186,0.686636,0.750819
2,CatBoostClassifier_PUL,0.680338,0.722577,0.668944,0.692128
3,XGBClassifier_PUL,0.646853,0.692956,0.638731,0.655183


**Вывод: исходя из полученных результатов, на этом наборе данных random negative sampling как метод показал себя очень даже неплохо. Разницу в работе моделей вероятно можно объяснить тем, что данные не были подготовлены и разные модели по-разному на это среагировали.**